# Lab 6: Hosting Strands Agents with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this lab we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. We will provide examples using Amazon Bedrock models, although other non-Bedrock models such as Azure OpenAI and Gemini can also be a valid choice.

### Lab Details

| Information        | Details                                                                          |
| :----------------- | :------------------------------------------------------------------------------- |
| Lab type           | Conversational                                                                   |
| Agent type         | Single                                                                           |
| Agentic Framework  | Strands Agents                                                                   |
| LLM model          | Anthropic Claude Sonnet 4                                                        |
| Lab components     | Hosting agent on AgentCore Runtime. Using Strands Agent and Amazon Bedrock Model |
| Lab vertical       | Customer Support                                                                 |
| Example complexity | Easy                                                                             |
| SDK used           | Amazon BedrockAgentCore Python SDK and boto3                                     |

### Lab Architecture

In this Lab we will describe how to deploy an existing agent to AgentCore Runtime.

For demonstration purposes, we will use a Strands Agent using Amazon Bedrock models.

In our example we will use a previously defined Agent with some built-in Google calendar tools, AgentCore Identity access management, AgentCore Observability, using AgentCore Memory, and connected through AgentCore Gateway to additional customer-related lambda tools.

### Lab Key Features

- Hosting Agents on Amazon Bedrock AgentCore Runtime
- Using Amazon Bedrock models
- Using Strands Agents


## Prerequisites

To execute this tutorial you will need:

- Python 3.10+
- AWS credentials
- Amazon Bedrock AgentCore SDK
- Strands Agents
- **Previously completed Labs 1 through 5**

**Note**: the modules implemented using other AgentCore Services, covered in previous Labs, are stored in `customer_support_agent` folder, required for our agent to operate in full.


In [1]:
# Import some utilities:
import os
import boto3

from customer_support_agent.utils import get_ssm_parameter

## Creating your agents and experimenting locally

With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. As a secure, serverless runtime purpose-built for deploying and scaling dynamic AI agents and tools, AgentCore Runtime will complement other previously defined AgentCore Services to obtain one modular Customer Support Agent solution:

<div style="text-align:left">
    <img src="images/architecture_global.png" width="100%"/>
</div>


## Preparing your agent for deployment on AgentCore Runtime

To deploy our Agent to AgentCore Runtime, we need to:

- Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
- Initialize the App in our code with `app = BedrockAgentCoreApp()`
- Decorate the invocation function with the `@app.entrypoint` decorator
- Let the AgentCore Runtime control the running of the agent with `app.run()`

### Incorporating AgentCore App with Strands Agents

Let's create our AgentCore App featuring previously built customer support Strands Agent. We will also save locally this script to be deployed later on


In [ ]:
%%writefile agent.py
import asyncio
import logging
import os

from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Reuse previously developed modules and services:
from customer_support_agent.access_token import get_gateway_access_token
from customer_support_agent.agent_task import agent_task
from customer_support_agent.context import CustomerSupportContext
from customer_support_agent.streaming_queue import StreamingQueue
from customer_support_agent.utils import get_ssm_parameter

# Environment flags
os.environ["STRANDS_OTEL_ENABLE_CONSOLE_EXPORT"] = "true"
os.environ["STRANDS_TOOL_CONSOLE_MODE"] = "enabled"

# The Bedrock Knowledge Base our agent will have access to
os.environ["KNOWLEDGE_BASE_ID"] = get_ssm_parameter(
    "/app/customersupport/knowledge_base/knowledge_base_id"
)

# Logging setup
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Bedrock app and global agent instance
app = BedrockAgentCoreApp()


@app.entrypoint
async def invoke(payload, context):
    if not CustomerSupportContext.get_response_queue_ctx():
        CustomerSupportContext.set_response_queue_ctx(StreamingQueue())

    if not CustomerSupportContext.get_gateway_token_ctx():
        CustomerSupportContext.set_gateway_token_ctx(await get_gateway_access_token())

    user_message = payload["prompt"]
    actor_id = payload["actor_id"]

    session_id = context.session_id

    if not session_id:
        raise Exception("Context session_id is not set")

    task = asyncio.create_task(
        agent_task(
            user_message=user_message,
            session_id=session_id,
            actor_id=actor_id,
        )
    )

    response_queue = CustomerSupportContext.get_response_queue_ctx()

    async def stream_output():
        async for item in response_queue.stream():
            yield item
        await task  # Ensure task completion

    return stream_output()


if __name__ == "__main__":
    app.run()

Overwriting agent.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

- Creates an HTTP server that listens on the port 8080
- Implements the required `/invocations` endpoint for processing the agent's requirements
- Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
- Handles proper content types and response formats
- Manages error handling according to the AWS standards


## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent.

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this Lab you will use the AgentCore Starter Toolkit Python SDK to easily package your artifacts and deploy them to AgentCore Runtime.


### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>


In [3]:
import json

cognito_user_pool_id = get_ssm_parameter("/app/customersupport/agentcore/userpool_id")

# Create an additional Cognito Client to provide the authorization token:
!bash set_auth_token.sh {cognito_user_pool_id}

with open("client.json") as f:
    cognito_client_id = json.load(f)["UserPoolClient"]["ClientId"]

with open("auth.json") as f:
    bearer_token = json.load(f)["AuthenticationResult"]["AccessToken"]

Using AWS Region: us-west-2
Using User Pool: us-west-2_OWAfwDppo

{
  "User": {
    "Username": "189123a0-4031-70c4-ad19-798d1dcc4e19",
    "Attributes": [
      {
        "Name": "email",
        "Value": "testuser@example.com"
      },
      {
        "Name": "sub",
        "Value": "189123a0-4031-70c4-ad19-798d1dcc4e19"
      }
    ],
    "UserCreateDate": "2025-07-31T13:11:50.429000+02:00",
    "UserLastModifiedDate": "2025-07-31T13:11:50.429000+02:00",
    "Enabled": true,
    "UserStatus": "FORCE_CHANGE_PASSWORD"
  }
}
Successfully extracted and saved access token to auth.json


In [4]:
from bedrock_agentcore_starter_toolkit import Runtime

boto_session = boto3.session.Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    # Previosly defined agent invokation script:
    entrypoint="agent.py",
    # The pre-defined IAM role for the AgentCore Runtime to be able to host our agent:
    execution_role=get_ssm_parameter("/app/customersupport/agentcore/runtime_iam_role"),
    auto_create_ecr=True,
    # Define a set of python requirements for our agent:
    requirements_file="requirements.txt",
    region=region,
    # Let's give our agent a name:
    agent_name="customersupport_agent",
    # Ensure inbound authorization (Lab 3):
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": get_ssm_parameter(
                "/app/customersupport/agentcore/cognito_discovery_url"
            ),
            "allowedClients": [cognito_client_id],
        }
    },
)
response

Entrypoint parsed: file=/Users/sielskyi/Documents/AWS/learning/amazon-bedrock-agentcore-samples-dev/01-tutorials/07-AgentCore-E2E/06-AgentCore-runtime-host-your-agent/agent.py, bedrock_agentcore_name=agent
Configuring BedrockAgentCore agent: customersupport_agent
Generated Dockerfile: /Users/sielskyi/Documents/AWS/learning/amazon-bedrock-agentcore-samples-dev/01-tutorials/07-AgentCore-E2E/06-AgentCore-runtime-host-your-agent/Dockerfile
Generated .dockerignore: /Users/sielskyi/Documents/AWS/learning/amazon-bedrock-agentcore-samples-dev/01-tutorials/07-AgentCore-E2E/06-AgentCore-runtime-host-your-agent/.dockerignore
Keeping 'customersupport_agent' as default agent
Bedrock AgentCore configured: /Users/sielskyi/Documents/AWS/learning/amazon-bedrock-agentcore-samples-dev/01-tutorials/07-AgentCore-E2E/06-AgentCore-runtime-host-your-agent/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/sielskyi/Documents/AWS/learning/amazon-bedrock-agentcore-samples-dev/01-tutorials/07-AgentCore-E2E/06-AgentCore-runtime-host-your-agent/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/sielskyi/Documents/AWS/learning/amazon-bedrock-agentcore-samples-dev/01-tutorials/07-AgentCore-E2E/06-AgentCore-runtime-host-your-agent/Dockerfile'), dockerignore_path=PosixPath('/Users/sielskyi/Documents/AWS/learning/amazon-bedrock-agentcore-samples-dev/01-tutorials/07-AgentCore-E2E/06-AgentCore-runtime-host-your-agent/.dockerignore'), runtime='Finch', region='us-west-2', account_id='528757807227', execution_role='arn:aws:iam::528757807227:role/CustomerSupportStackInfra-RuntimeAgentCoreRole-MXI4b5F5dPsq', ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime by means of CodeBuild

For the sake of this lab, to avoid additional `Docker` dependencies, we will delegate our Agent's image build and deployment taks to the AWS CodeBuild project. It will create the Amazon ECR repository and the AgentCore Runtime for us.

<div style="text-align:left">
    <img src="images/deploy_with_codebuild.png" width="75%"/>
</div>


In [5]:
launch_result = agentcore_runtime.launch(use_codebuild=True)

Starting CodeBuild ARM64 deployment for agent 'customersupport_agent' to account 528757807227 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: customersupport_agent
✅ ECR repository available: 528757807227.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-customersupport_agent
Using execution role from config: arn:aws:iam::528757807227:role/CustomerSupportStackInfra-RuntimeAgentCoreRole-MXI4b5F5dPsq


✅ Reusing existing ECR repository: 528757807227.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-customersupport_agent


✅ Execution role validation passed: arn:aws:iam::528757807227:role/CustomerSupportStackInfra-RuntimeAgentCoreRole-MXI4b5F5dPsq
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: customersupport_agent
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-080f430d2c
Reusing existing CodeBuild execution role: arn:aws:iam::528757807227:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-080f430d2c
Created S3 bucket: bedrock-agentcore-codebuild-sources-528757807227-us-west-2
Using .dockerignore with 44 patterns
Uploaded source to S3: customersupport_agent/20250731-111202.zip
Created CodeBuild project: bedrock-agentcore-customersupport_agent-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 5.2s
🔄 PROVISIONING started (total: 5s)
✅ PROVISIONING completed in 10.4s
🔄 PRE_BUILD started (total: 16s)
✅ PRE_BUILD completed in 10.4s
🔄 BUILD s

### Checking for the AgentCore Runtime Status

Now that we've deployed the AgentCore Runtime, let's check for it's deployment status


In [6]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]
end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]
    print(status)
status

Retrieved Bedrock AgentCore status for: customersupport_agent


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/deploy_with_codebuild_and_test.png" width=75%"/>
</div>


#### Invoke agent via AgentCore Starter Toolkit Runtime object

We can now invoke our agent and process the results.


In [7]:
from IPython.display import Markdown, display


def cleanup_streamed_response(response: str) -> str:
    """
    Since the response is meant to be streamed,
        let's process it to prettify the final response.
    """

    return response.replace('"\n"', "").replace("\\n", "\n").strip('"')


invoke_response = agentcore_runtime.invoke(
    {
        "prompt": "I have a Gaming Console Pro device, I want to check my warranty status, warranty serial number is MNO33333333.",
        "actor_id": "DEFAULT",
    },
    bearer_token=bearer_token,
)
Markdown(cleanup_streamed_response(invoke_response["response"]))

Invoking BedrockAgentCore agent 'customersupport_agent' via cloud endpoint


I'll help you check the warranty status for your Gaming Console Pro device with serial number MNO33333333.I've checked the warranty status for your Gaming Console Pro device. Here's what I found:

**Warranty Status Information**
- **Product:** Gaming Console Pro
- **Serial Number:** MNO33333333
- **Purchase Date:** November 25, 2023
- **Warranty End Date:** November 25, 2024
- **Status:** ❌ **Expired** (249 days ago)

**Original Coverage Details:**
Your warranty covered controller issues, overheating protection, and hard drive replacement.

**What's Next:**
Since your warranty has expired, you may still have options:
- Extended warranty options may be available
- We can provide repair service pricing for any issues you're experiencing

If you're having any specific issues with your Gaming Console Pro or would like information about extended warranty options or repair services, please let me know and I'll be happy to help you further!

### Invoking AgentCore Runtime with HTTP client

Now that your AgentCore Runtime was created you can invoke it with the `requests` python library to send the authorization token as well (since `boto3` does not support such functionality).


In [8]:
import uuid
import urllib
import requests

agent_arn = launch_result.agent_arn

escaped_arn = urllib.parse.quote(agent_arn, safe="")
url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_arn}/invocations"

headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json",
    "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": str(uuid.uuid4()),
}

try:
    response = requests.post(
        url,
        params={"qualifier": "DEFAULT"},
        headers=headers,
        json={
            "prompt": "What are the warranty support guidelines?",
            "actor_id": "DEFAULT",
        },
        timeout=100,
        stream=True,
    )
except requests.exceptions.RequestException as e:
    print("Failed to invoke agent endpoint:", str(e))
    raise

# Stream response:
last_data = False
content = []
for line in response.iter_lines(chunk_size=1):
    if line:
        line = line.decode("utf-8")
        if line.startswith("data: "):
            last_data = True
            line = line[6:].replace('"', "")
            print(line, end="")
            content.append(line)
        elif line:
            if last_data:
                line = line.replace('"', "")
                print("\n" + line, end="")
                content.append(line)
            last_data = False

display(Markdown(cleanup_streamed_response("".join(content))))

I'll help you find information about our warranty support guidelines. Let me search our knowledge base for the most current warranty support policies and procedures.I apologize, but I'm currently unable to access our detailed warranty support guidelines through our system. However, I can provide you with some general information about warranty support based on what I know about your Gaming Console Pro device.\n\nSince you own a Gaming Console Pro with serial number MNO33333333 that is currently out of warranty (expired on November 25, 2024), here's what I can tell you about typical warranty support:\n\n**General Warranty Support Information:**\n\n1. **Original Warranty Coverage** - Your Gaming Console Pro's original warranty covered:\n   - Controller issues\n   - Overheating protection\n   - Hard drive replacement\n\n2. **Post-Warranty Options** - Even though your device is out of warranty, you may still have options for:\n   - Paid repair services\n   - Component replacement at cost\n

I'll help you find information about our warranty support guidelines. Let me search our knowledge base for the most current warranty support policies and procedures.I apologize, but I'm currently unable to access our detailed warranty support guidelines through our system. However, I can provide you with some general information about warranty support based on what I know about your Gaming Console Pro device.

Since you own a Gaming Console Pro with serial number MNO33333333 that is currently out of warranty (expired on November 25, 2024), here's what I can tell you about typical warranty support:

**General Warranty Support Information:**

1. **Original Warranty Coverage** - Your Gaming Console Pro's original warranty covered:
   - Controller issues
   - Overheating protection
   - Hard drive replacement

2. **Post-Warranty Options** - Even though your device is out of warranty, you may still have options for:
   - Paid repair services
   - Component replacement at cost
   - Diagnostic services

3. **Common Support Guidelines** typically include:
   - Proof of purchase requirements
   - Device condition assessments
   - Available repair vs. replacement options
   - Estimated repair timeframes and costs

Would you like me to check the current warranty status of your specific device to see if there are any available support options, or do you have a specific warranty-related question I can help address?

## Congratulations!

You have successfully deployed and invoked your first AgentCore Runtime!


## Cleanup (Optional)

Let's now clean up the created AgentCore Runtime


In [9]:
launch_result.agent_id, launch_result.ecr_uri, launch_result.codebuild_id

('customersupport_agent-D1WtvKBDZt',
 '528757807227.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-customersupport_agent',
 'bedrock-agentcore-customersupport_agent-builder:76353753-7f37-412f-82f4-a4a2249ab755')

In [10]:
agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)
response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)
print(response)

ecr_client = boto3.client("ecr", region_name=region)
response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split("/")[1], force=True
)
print(response)

codebuild_client = boto3.client("codebuild")

codebuild_project = launch_result.codebuild_id.split(":")[0]

codebuild_bucket = codebuild_client.batch_get_projects(names=[codebuild_project])[
    "projects"
][0]["source"]["location"].split("/")[0]

response = codebuild_client.delete_project(name=codebuild_project)
print(response)

s3 = boto3.resource("s3")
bucket = s3.Bucket(codebuild_bucket)
bucket.object_versions.delete()

s3_client = boto3.client("s3")
response = s3_client.delete_bucket(Bucket=codebuild_bucket)
print(response)

{'ResponseMetadata': {'RequestId': 'a543828f-118f-47df-b90f-e2fa5bb14adb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 31 Jul 2025 11:15:42 GMT', 'content-type': 'application/json', 'content-length': '21', 'connection': 'keep-alive', 'x-amzn-requestid': 'a543828f-118f-47df-b90f-e2fa5bb14adb', 'x-amzn-remapped-x-amzn-requestid': '249802f0-dcf7-46a0-a33b-ecc5230a0ef4', 'x-amzn-remapped-content-length': '21', 'x-amzn-remapped-connection': 'keep-alive', 'x-amz-apigw-id': 'Okl-zHXWvHcEe2A=', 'x-amzn-trace-id': 'Root=1-688b505e-13444a737e6ea60063ecdf80', 'x-amzn-remapped-date': 'Thu, 31 Jul 2025 11:15:42 GMT'}, 'RetryAttempts': 0}, 'status': 'DELETING'}
{'repository': {'repositoryArn': 'arn:aws:ecr:us-west-2:528757807227:repository/bedrock-agentcore-customersupport_agent', 'registryId': '528757807227', 'repositoryName': 'bedrock-agentcore-customersupport_agent', 'repositoryUri': '528757807227.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-customersupport_agent', 'createdAt': date

In [11]:
# Delete local AgentCore configuration:
os.unlink("Dockerfile")
os.unlink(".dockerignore")
os.unlink(".bedrock_agentcore.yaml")
os.unlink("client.json")
os.unlink("auth.json")